In [2]:
import torch
import torchvision.models as models
import torch.nn as nn
from PIL import Image
import torchvision.transforms as transforms
import os
import pandas as pd

# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pretrained ResNet18 모델 불러오기
model = models.resnet18(pretrained=False)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# 저장된 모델 상태 로드
model.load_state_dict(torch.load('resnet18_shoes_classifier.pth'))

# 모델을 GPU로 전송
model = model.to(device)
model.eval()

# 이미지 전처리 함수 정의
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # ImageNet 정규화 값
    ])
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # 배치 차원을 추가
    return image

# 클래스 라벨
class_labels = ['etc', 'full']

# 이미지 폴더 경로
image_folder_path = './Airforce_white'

# 예측 결과 저장할 리스트 초기화
results = []
# 폴더 내 모든 이미지에 대해 예측 수행
for image_name in os.listdir(image_folder_path):
    image_path = os.path.join(image_folder_path, image_name)
    if image_path.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif')):
        image = preprocess_image(image_path)
        image = image.to(device)
        
        # 예측 수행
        with torch.no_grad():
            outputs = model(image)
            _, predicted = torch.max(outputs, 1)
        
        # 예측된 라벨 저장
        predicted_class = class_labels[predicted.item()]
        results.append({'file_name': image_name, 'label': predicted_class})

# 결과를 DataFrame으로 변환
df = pd.DataFrame(results)

# CSV 파일로 저장
df.to_csv('airforce_white_labels.csv', index=False)


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import datetime
import matplotlib.pyplot as plt
import collections as cc
import seaborn as sns
from collections import Counter
import re
from bs4 import BeautifulSoup
from konlpy.tag import Okt


data1_dir = "../data1.csv"
data2_dir = "../data2.csv"
data3_dir = "../data3.csv"
data4_dir = "../data4.csv"
### 전처리 완료 후 에어포스에 대해서 처리한 데이터 -> processed_data
data_dir='../Airfocre_tot.csv'
label_dir='../airforce_white_labels.csv'

In [3]:
import os
import shutil
import pandas as pd

# CSV 파일 로드
csv_file_path = 'airforce_white_labels.csv'
df = pd.read_csv(csv_file_path)

# 이미지 폴더 경로
image_folder_path = './Airforce_white'

# Full 및 etc 폴더 생성
full_folder = os.path.join(image_folder_path, 'Full')
etc_folder = os.path.join(image_folder_path, 'etc')

os.makedirs(full_folder, exist_ok=True)
os.makedirs(etc_folder, exist_ok=True)

# CSV 파일에 따라 이미지 분류
for index, row in df.iterrows():
    image_name = row['file_name']
    label = row['label']
    
    src_path = os.path.join(image_folder_path, image_name)
    
    if label == 'full':
        dest_path = os.path.join(full_folder, image_name)
    elif label == 'etc':
        dest_path = os.path.join(etc_folder, image_name)
    else:
        continue
    
    # 이미지 이동
    shutil.move(src_path, dest_path)

print('이미지가 라벨에 따라 분류되었습니다.')


이미지가 라벨에 따라 분류되었습니다.


In [5]:
data_dir = "./Airforce_white_fullimage_score.csv"

In [6]:
df = pd.read_csv(data_dir)


In [8]:
df.min()

Unnamed: 0    01511419fe76796d53984351b2237017_2.jpg
Score                                       0.236543
dtype: object

In [9]:
import mrcnn.model as modellib
from mrcnn import visualize

ModuleNotFoundError: No module named 'mrcnn'

In [1]:
import os
import numpy as np
import skimage.io
import tensorflow as tf
from tensorflow import keras
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config

# Mask R-CNN 설정
class InferenceConfig(Config):
    NAME = "shoes_detection"
    NUM_CLASSES = 1 + 1  # Background + 신발 클래스
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.6  # 최소 신뢰도

config = InferenceConfig()

# 모델 디렉터리와 가중치 파일 경로
MODEL_DIR = "logs"
WEIGHTS_PATH = "path_to_trained_weights.h5"  # 훈련된 가중치 파일 경로

# Mask R-CNN 모델 초기화
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# 가중치 로드
model.load_weights(WEIGHTS_PATH, by_name=True)

# 이미지 로드
image = skimage.io.imread('path_to_image')

# 이미지에서 객체 감지
results = model.detect([image], verbose=1)

# 결과 시각화
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            ['BG', 'shoe'], r['scores'])


ModuleNotFoundError: No module named 'mrcnn'

In [4]:
import os
import numpy as np
import skimage.io
import tensorflow as tf
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.config import Config

# Mask R-CNN 설정
class InferenceConfig(Config):
    NAME = "shoes_detection"
    NUM_CLASSES = 1 + 1  # Background + 신발 클래스
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.6  # 최소 신뢰도

config = InferenceConfig()

# 모델 디렉터리와 가중치 파일 경로
MODEL_DIR = "logs"
WEIGHTS_PATH = "path_to_trained_weights.h5"  # 훈련된 가중치 파일 경로

# Mask R-CNN 모델 초기화
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# 가중치 로드
model.load_weights(WEIGHTS_PATH, by_name=True)

# 이미지 로드
image = skimage.io.imread('path_to_image')

# 이미지에서 객체 감지
results = model.detect([image], verbose=1)

# 결과 시각화
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            ['BG', 'shoe'], r['scores'])


ModuleNotFoundError: No module named 'keras.engine'